# MOD16A2数据重采样

尝试对MOD16A2数据进行重采样。这里使用广泛被采用的最大值合成法。

## 最大值合成法

记$\{PET_i\}$为一个栅格中的降雨量时间序列，则在该时间范围内，这一栅格中的降雨量值可以用序列中的最大值$\mathrm{max}(\{PET_i\})$代替。

$$ PET=\mathrm{max}(\{PET_i\}) $$

扩展到二维形式，使用循环解决
```python
import numpy as np
PET_sequence = np.array([[[PET_tij for j in range(0,j)]for i in range(0,i)]for t in range(0,t)])

PET = []
for t in range(0,t):
    PET_ij = []
    for i in range(0,i):
        PET_i = []
        for j in range(0,j):
            PET_std = max([PET_sequence[i][j] for t in range(0,t)])
            PET_i.append(PET_std)
        PET_ij.append(PET_i)
    PET.append(PET_ij)

PET = np.array(PET)
```
使用了四重嵌套循环进行合成，效率相当的低。
能否进行一些优化？

可以使用`xarray`库[自带](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.max.html)的方法取最大值。`numpy`好像[也有](https://numpy.org/doc/stable/reference/generated/numpy.amax.html#numpy.amax)?

比如

~~~python
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 5]])
b = np.array([[4, 5, 1], [2, 3, 6], [7, 8, 9]])

np.amax([a, b], axis=0)
~~~


MODIS数据是8天一个样品，有必要根据日进行划分
根据时间间隔，将8天的采样频率平均划分给12个月，如下表：

| **Month** | **Start_date** | **End_date** | **Length** | **Start_date_filename** | **End_date_filename** |
|-----------|----------------|--------------|------------|-------------------------|------------------------|
| **1**     | 1              | 32           | 31         | 1                       | 25                     |
| **2**     | 33             | 56           | 23         | 33                      | 49                     |
| **3**     | 57             | 88           | 31         | 57                      | 81                     |
| **4**     | 89             | 120          | 31         | 89                      | 113                    |
| **5**     | 121            | 152          | 31         | 121                     | 145                    |
| **6**     | 153            | 184          | 31         | 153                     | 177                    |
| **7**     | 185            | 216          | 31         | 185                     | 209                    |
| **8**     | 209            | 240          | 31         | 209                     | 233                    |
| **9**     | 241            | 272          | 31         | 241                     | 265                    |


In [14]:
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import shapely
import rioxarray as rxr
import rasterio
from affine import Affine
import multiprocessing


定义生成文件名字典的函数

In [15]:
def generate_filename_map(year:str)->dict:
    
    def generate_filename(year:str,date:str)->str:
        preifx = 'MOD16A2GF.006_PET_500m_doy'
        suffix = '_aid0001.tif'
        return preifx+year+date+suffix
    
    def calc_days(start:int,end:int)->list:
        length = end-start
        counts = length//8
        days = []
        for i in range(counts):
            days.append(start+i*8)
        days.append(end)
        return days
    
    months = [1,2,3,4,5,6,7,8,9,10,11,12]
    months_start_date_lst = [1,33,57,89,121,153,185,209,241,273,305,337]
    months_end_date_lst = [25,49,81,113,145,177,209,233,265,297,329,361]

    filename_map = {}

    for i in range(len(months)):
        start = months_start_date_lst[i]
        end = months_end_date_lst[i]
        days = calc_days(start,end)
        filename_map[months[i]] = []
        for day in days:
            filename_map[months[i]].append(generate_filename(year,str(day).zfill(3)))

    return filename_map

测试一下求最大值

In [16]:
base_dir = '../datas/MODIS/MODIS16A2GF/PET/'

test_frame_0 = xr.open_dataset(r"../datas/MODIS/MODIS16A2GF/PET/MOD16A2GF.006_PET_500m_doy2000001_aid0001.tif").squeeze("band",drop=True)


test_frame_1 = xr.open_dataset(r"../datas/MODIS/MODIS16A2GF/PET/MOD16A2GF.006_PET_500m_doy2000009_aid0001.tif").squeeze("band",drop=True)


test_frame_2 = xr.open_dataset(r"../datas/MODIS/MODIS16A2GF/PET/MOD16A2GF.006_PET_500m_doy2000017_aid0001.tif").squeeze("band",drop=True)


test_frame = xr.concat([test_frame_0,test_frame_1,test_frame_2],dim="time",data_vars=["band_data"])

In [17]:
test_frame=test_frame.max(dim="time")

MODIS16A2的空间分辨率为500m，为与降雨量数据精度匹配，需要将其重采样至1km。

In [18]:
test_frame

<xarray.Dataset>
Dimensions:      (x: 1681, y: 1921)
Coordinates:
  * x            (x) float64 114.0 114.0 114.0 114.0 ... 121.0 121.0 121.0 121.0
  * y            (y) float64 40.0 40.0 39.99 39.99 ... 32.01 32.01 32.01 32.0
    spatial_ref  int64 0
Data variables:
    band_data    (y, x) float32 8.6 8.6 8.6 8.6 8.6 ... 12.6 12.6 12.5 12.4 12.5

这是降雨量的数据

In [19]:
precip_test = xr.open_dataset(r"../outputs/precip-cutted/_2000_2002_1.tif")
precip_test = precip_test.squeeze("band",drop=True)
precip_test.band_data.to_numpy().shape

(960, 840)

将PET数据使用双线性重采样到$1km\times 1km$分辨率

In [20]:
test_frame_resampled= test_frame.rio.reproject(test_frame.rio.crs,shape=(960,840),resampling=rasterio.enums.Resampling.bilinear)
test_frame_resampled.band_data.to_numpy().shape

(960, 840)

分别读取每一年的对应数据，并且合成单独的文件

理论上来说，`rioxarray`库会在保存到geotiff文件时自动加入`band`条带。这是正常现象。

In [21]:
def resample_pet(year:str)->None:
    
    filename_map = generate_filename_map(year)
    
    for month in filename_map.keys():
        print("Processing month: ",month)
        month_frames = []
        for filename in filename_map[month]:
            print("Processing file: ",filename)
            frame = xr.open_dataset(base_dir+filename)
            frame = frame.squeeze("band",drop=True)
            month_frames.append(frame)
        month_frames = xr.concat(month_frames,dim="time",data_vars=["band_data"])
        month_frames_max_prod=month_frames.max(dim="time")*len(filename_map[month])
        # resampling
        month_frames_max_resampled = month_frames_max_prod.rio.reproject(month_frames_max_prod.rio.crs,shape=(960,840),resampling=rasterio.enums.Resampling.bilinear)

        month_frames_max_resampled.rio.to_raster("../outputs/pet-resampled/"+year+"_"+str(month).zfill(2)+".tif")
        print("Month: ",month," done!")



这是使用多进程的方式

In [22]:
process_pool = multiprocessing.Pool(30)

for i in range(2000,2016):
    process_pool.apply_async(resample_pet,args=(str(i),))

process_pool.close()
process_pool.join()

Processing month: Processing month: Processing month: Processing month: Processing month: Processing month: Processing month: Processing month: Processing month: Processing month: Processing month: Processing month: Processing month:  Processing month: Processing month:    Processing month:         1  11 1 11111111
1
1
1
1







Processing file: 
Processing file: 
Processing file: Processing file: 

Processing file: Processing file: Processing file: Processing file: Processing file: Processing file: Processing file: Processing file:  Processing file:  Processing file:   Processing file: Processing file:         MOD16A2GF.006_PET_500m_doy2000001_aid0001.tifMOD16A2GF.006_PET_500m_doy2001001_aid0001.tif  MOD16A2GF.006_PET_500m_doy2002001_aid0001.tifMOD16A2GF.006_PET_500m_doy2003001_aid0001.tif  MOD16A2GF.006_PET_500m_doy2004001_aid0001.tifMOD16A2GF.006_PET_500m_doy2005001_aid0001.tifMOD16A2GF.006_PET_500m_doy2007001_aid0001.tifMOD16A2GF.006_PET_500m_doy2006001_aid0001.tifMOD16A2GF.006_PE

/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/rioxarray/raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/rioxarray/raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(
/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/rioxarray/raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


Month:  1Month:    done!1
 Processing month:   done!2

Processing month: Processing file:   2MOD16A2GF.006_PET_500m_doy2001033_aid0001.tif

Processing file:  MOD16A2GF.006_PET_500m_doy2000033_aid0001.tif
Month:  1  done!Processing file: 
 Processing month: MOD16A2GF.006_PET_500m_doy2001041_aid0001.tif 


/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/rioxarray/raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


2
Processing file:  MOD16A2GF.006_PET_500m_doy2005033_aid0001.tif
Processing file:  

...

Month:  12  done!
Month:  12  done!


/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/rioxarray/raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


Month:  12  done!


/home/zyzh0/anaconda3/envs/gis/lib/python3.9/site-packages/rioxarray/raster_writer.py:132: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


Month:  12  done!
